In [1]:
import random
from math import ceil
from typing import Dict, Tuple
from typing import List, Set

import torch
from torch import Tensor
from torch import nn
from torch.utils.data import Dataset, DataLoader

from graph import Graph
from part import Part
from preprocess import create_family_id_mapping
from utils import load_graphs

In [2]:
graphs = load_graphs()
mapping = create_family_id_mapping(graphs)

seed = 5
random = random.Random(seed)
random.shuffle(graphs)

In [3]:
def create_features(graph: Graph, dense_family_id_mapping: Dict) -> Tuple[List[Tensor], List[Tensor]]:
    num_different_family_ids = len(dense_family_id_mapping)

    # Create a tensor for existing nodes and count their occurrences
    all_nodes_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
    nodes = graph.get_nodes()
    edges = graph.get_edges()

    feature_tensors = []
    target_tensors = []

    for node in nodes:
        dense_family_id = dense_family_id_mapping[int(node.get_part().get_family_id())]
        all_nodes_tensor[dense_family_id] += 1.0

        target_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
        for neighbour_node in edges[node]:
            neighbour_node_dense_family_id = dense_family_id_mapping[int(neighbour_node.get_part().get_family_id())]
            target_tensor[neighbour_node_dense_family_id] += 1
        target_tensors.append(target_tensor)

    for node in nodes:
        dense_family_id = dense_family_id_mapping[int(node.get_part().get_family_id())]
        given_node_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
        given_node_tensor[dense_family_id] = 1

        feature_tensors.append(torch.cat((all_nodes_tensor, given_node_tensor), dim=-1))

    return feature_tensors, target_tensors
    # return list(zip(feature_tensors, target_tensors))

In [ ]:
# split data into train, val and test - they are already shuffled

train_upper = ceil(0.8 * len(graphs))
val_upper = ceil(0.9 * len(graphs))

train_graphs = graphs[0:train_upper]
val_graphs = graphs[train_upper:val_upper]
test_graphs = graphs[val_upper:len(graphs) + 1]


class GraphDataset(Dataset):
    def __init__(self, graphs: List[Graph]):
        x_list = []
        y_list = []
        for graph in graphs:
            x_per_graph, y_per_graph = create_features(graph, mapping)
            x_list += x_per_graph
            y_list += y_per_graph
        self.x_tensor = torch.stack(x_list)
        self.y_tensor = torch.stack(y_list)
        self.n_samples = self.y_tensor.shape[0]

    def __getitem__(self, index):
        return self.x_tensor[index], self.y_tensor[index]

    def __len__(self):
        return self.n_samples


batch_size = 100
train_loader = DataLoader(dataset=GraphDataset(train_graphs), batch_size=batch_size,
                          shuffle=False)  #TODO: add num_workers=8
val_loader = DataLoader(dataset=GraphDataset(val_graphs), batch_size=batch_size,
                         shuffle=False)  #TODO: add num_workers=8

In [5]:
input_size = 2 * len(mapping)
hidden_size = 3 * len(mapping)
output_size = len(mapping)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        # self.fc2 = nn.Linear(..., ...)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


learning_rate = 0.0005
num_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
n_total_steps = len(train_loader)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()  #nn.BCELoss()
train_loss = []

for epoch in range(num_epochs):
    print(f"epoch {epoch}")
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        y_pred = model(x_train)
        loss = loss_fn(y_pred, y_train)
        train_accuracy = torch.sum(y_train == y_pred.round()) / torch.numel(y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"loss {loss}")
            print(f"train_accuracy {train_accuracy}")

    with torch.no_grad():
        val_accuracies = []
        for i, (x_val, y_val) in enumerate(val_loader):
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            y_pred = model(x_val)
            val_accuracy = torch.sum(y_val == y_pred.round()) / torch.numel(y_val)
            val_accuracies.append(val_accuracy)
        print(f"validation acc: {torch.mean(torch.stack(val_accuracies))}")
# plt.plot(train_loss)
# plt.show()

epoch 0
loss 0.25635042786598206
train_accuracy 0.4517391324043274
loss 0.023113489151000977
train_accuracy 0.9835869669914246
loss 0.01993025839328766
train_accuracy 0.9833695888519287
loss 0.019973130896687508
train_accuracy 0.9856522083282471
loss 0.01798643358051777
train_accuracy 0.9873913526535034
loss 0.01799234189093113
train_accuracy 0.9878261089324951
loss 0.014070602133870125
train_accuracy 0.9889130592346191
validation acc: 0.9889240860939026
epoch 1
loss 0.01159029733389616
train_accuracy 0.9908695816993713
loss 0.01155709195882082
train_accuracy 0.9900000095367432
loss 0.011953765526413918
train_accuracy 0.990760862827301
loss 0.013188793323934078
train_accuracy 0.990217387676239
loss 0.011146407574415207
train_accuracy 0.9908695816993713
loss 0.01040367316454649
train_accuracy 0.9908695816993713
loss 0.00917848851531744
train_accuracy 0.9919565320014954
validation acc: 0.9916764497756958
epoch 2
loss 0.007789890747517347
train_accuracy 0.9936956763267517
loss 0.007262596

In [6]:
from evaluation import __generate_part_list_permutations
import numpy as np

def edge_accuracy(predicted_graph: Graph, target_graph: Graph) -> int:
    """
    Returns the number of correct predicted edges.
    :param predicted_graph:
    :param target_graph:
    :return:
    """
    assert len(predicted_graph.get_nodes()) == len(target_graph.get_nodes()), 'Mismatch in number of nodes.'
    assert predicted_graph.get_parts() == target_graph.get_parts(), 'Mismatch in expected and given parts.'

    best_score = 0

    # Determine all permutations for the predicted graph and choose the best one in evaluation
    perms: List[Tuple[Part]] = __generate_part_list_permutations(predicted_graph.get_parts())
    # Determine one part order for the target graph
    target_parts_order = perms[0]
    target_adj_matrix = target_graph.get_adjacency_matrix(target_parts_order)

    if len(perms) > 80_000:
        return 0
    for perm in perms:
        predicted_adj_matrix = predicted_graph.get_adjacency_matrix(perm)
        score = np.sum(predicted_adj_matrix == target_adj_matrix)
        best_score = max(best_score, score)

    return best_score

# from evaluation import edge_accuracy

dense_family_id_to_family_id = {v: k for k, v in mapping.items()}
edge_accuracies = []
cpu_model = model.cpu()
with torch.no_grad():
    for index, test_graph in enumerate(test_graphs[:200]): #TODO remove :200
        # matrix dimensions: x = nodes_list index, y = dense family id
        predicted_adjacency_matrix = []
        predicted_graph_count = 0
        predicted_graph = Graph()
        x, y = create_features(test_graph, mapping)
        for x_test, y_test in zip(x, y):
            y_test_pred = cpu_model(x_test)
            predicted_adjacency_matrix.append(y_test_pred)
        predicted_adjacency_matrix = torch.stack(predicted_adjacency_matrix)

        nodes = test_graph.get_nodes()
        nodes_list = list(nodes)
        node_count = len(nodes)
        added_nodes = set()
        last_rep_node_a_index = -1
        last_node_b_dense_family_id = -1
        predicted_graph_edge_count = 0
        while True:
            max_pos_tensor = (predicted_adjacency_matrix == torch.max(predicted_adjacency_matrix)).nonzero()
            rep_node_a_index = max_pos_tensor[0][0].item()
            node_b_dense_family_id = max_pos_tensor[0][1].item()
            if rep_node_a_index == last_rep_node_a_index and node_b_dense_family_id == last_node_b_dense_family_id:
                break
            last_rep_node_a_index = rep_node_a_index
            last_node_b_dense_family_id = node_b_dense_family_id
            node_b_family_id = dense_family_id_to_family_id[node_b_dense_family_id]
            node_a_family_id = int(nodes_list[rep_node_a_index].get_part().get_family_id())
            node_a_dense_family_id = mapping[node_a_family_id]

            # node_a = nodes_list[node_a_index]
            # node_a_candidates = list(filter(lambda node_a: int(node_a.get_part().get_family_id()) == node_a_family_id,nodes_list))
            # node_b_candidates = list(filter(lambda node_b: int(node_b.get_part().get_family_id()) == node_b_family_id,nodes_list))
            # if len(node_a_candidates) == len(node_b_candidates):
            #     candidate_pairs = zip(node_a_candidates,node_b_candidates)
            # else:
            for node_a_index, node_a in enumerate(nodes_list):
                if int(node_a.get_part().get_family_id()) == node_a_family_id:
                    for node_b_index, node_b in enumerate(nodes_list):
                        if int(node_b.get_part().get_family_id()) == node_b_family_id:
                            if not (node_a in added_nodes and node_b in added_nodes):
                                predicted_graph.add_undirected_edge(node_a.get_part(), node_b.get_part())
                                predicted_graph_edge_count += 1
                                added_nodes.add(node_a)
                                added_nodes.add(node_b)

                            # non-necessary performance optimization
                            predicted_adjacency_matrix[node_a_index][node_b_dense_family_id] = 0.0
                            predicted_adjacency_matrix[node_b_index][node_a_dense_family_id] = 0.0
            predicted_adjacency_matrix[rep_node_a_index][node_b_dense_family_id] = 0.0
            if predicted_graph_edge_count == node_count - 1:
                break
        edge_accuracies.append(edge_accuracy(test_graph, predicted_graph) / (node_count * node_count))
sum(edge_accuracies) / len(edge_accuracies)

0.9641430805678898

In [7]:
# Test accuracy of randomly generated trees
edge_accuracies = []
for test_graph in test_graphs[:100]:
    predicted_graph = Graph()
    nodes = test_graph.get_nodes()
    node_count = len(nodes)
    random_nodes_list = list(nodes)
    random.shuffle(random_nodes_list)
    node_a_index = 0
    node_b_index = 1

    for node_a_index in range(node_count - 1):
        existing_node_index = random.randint(0, node_a_index)
        existing_node = random_nodes_list[existing_node_index]
        new_node_index = node_a_index + 1
        new_node = random_nodes_list[new_node_index]
        predicted_graph.add_undirected_edge(existing_node.get_part(), new_node.get_part())

    edge_accuracies.append(edge_accuracy(test_graph, predicted_graph) / (node_count * node_count))

sum(edge_accuracies) / len(edge_accuracies)

0.6707963520575578